# Particle Swarm Optimisation

This notebook showcases how to use the built-in Particle Swarm Optimisation (PSO) algorithm.

In [11]:
using EvoLP
using Statistics
using OrderedCollections

For this example, we will use the Michalewicz function:

In [12]:
@doc michalewicz

```
michalewicz(x; m=10)
```

The **Michalewicz** function is a $d$-dimensional function with several steep valleys, where `m` controls the steepness. `m` is usually set at 10. For 2 dimensions, $x^* = [2.20, 1.57]$, with $f(x^*) = -1.8011$.

$$
f(x) = -\sum_{i=1}^{d}\sin(x_i) \sin^{2m}\left(\frac{ix_i^2}{\pi}\right)
$$


In this case we will use `d=2` and `m=10`, which is the default value implemented in EvoLP.

In PSO, we use _particles_. Each particle has a position and a velocity, and remembers the best position it has visited.

We can create a population of particles in multiple ways, but EvoLP provides 2 particle generators with random positions: either uniform or following a normal distribution.

Let's use the normal generator:

In [13]:
@doc normal_rand_particle_pop

```
normal_rand_particle_pop(n, μ, Σ; rng=Random.GLOBAL_RNG)
```

Generate a population of `n` [`Particle`](@ref) using a normal distribution with means `μ``and covariance`Σ`.

`μ` expects a vector of length *l* (i.e. number of dimensions) while `Σ` expects an *l x l* matrix of covariances.

# Examples

```julia
julia> normal_rand_particle_pop(3, [0, 0], [1 0; 0 1])
3-element Vector{Particle}:
 Particle([-0.6025996585348097, -1.0055548956861133], [0.0, 0.0], Inf, [-0.6025996585348097, -1.0055548956861133], Inf)
 Particle([-0.7562454555135321, 1.9490439959687778], [0.0, 0.0], Inf, [-0.7562454555135321, 1.9490439959687778], Inf)
 Particle([0.5687241357408321, -0.7406267072113427], [0.0, 0.0], Inf, [0.5687241357408321, -0.7406267072113427], Inf)
```


In [14]:
population = normal_rand_particle_pop(50, [0, 0], [1 0; 0 1])
first(population, 3)

3-element Vector{Particle}:
 Particle([0.4040308336567521, -1.0956708260389603], [0.0, 0.0], Inf, [0.4040308336567521, -1.0956708260389603], Inf)
 Particle([-0.5690754175927688, -1.9016076908148127], [0.0, 0.0], Inf, [-0.5690754175927688, -1.9016076908148127], Inf)
 Particle([0.9895469924697422, -1.2022337920748436], [0.0, 0.0], Inf, [0.9895469924697422, -1.2022337920748436], Inf)

Let's use the `Logbook` to save information about each iteration of the run:

In [15]:
@doc Logbook

```
Logbook(S::LittleDict)
```

A log for statistics intended for use on every iteration of an algorithm. The logbook is constructed from a `LittleDict` ordered dictionary which maps stat names (strings) to callables, such that *statname* $i$ can be computed from *callable* $i$.

The resulting `Logbook` contains:

  * `S::LittleDict`: The ordered dict of stat names and callables
  * `records::AbstractVector`: A vector of NamedTuples where each field is a statistic.


In [16]:
statnames = ["avg_fit", "median_fit", "best_fit"]
callables = [mean, median, minimum]

thedict = LittleDict(statnames, callables)
logbook = Logbook(thedict)

Logbook(LittleDict{AbstractString, Function, Vector{AbstractString}, Vector{Function}}("avg_fit" => Statistics.mean, "median_fit" => Statistics.median, "best_fit" => minimum), NamedTuple{(:avg_fit, :median_fit, :best_fit)}[])

We can now use the built-in algorithm:

In [17]:
@doc PSO

```
PSO(f, population, k_max; w=1, c1=1, c2=1)
PSO(logger::Logbook, f, population, k_max; w=1, c1=1, c2=1)
```

## Arguments

  * `f::Function`: Objective function to **minimise**.
  * `population::Vector{Particle}`: a list of [`Particle`](@ref) individuals.
  * `k_max::Integer`: number of iterations.

## Keywords

  * `w`: inertia weight. Optional, by default 1.
  * `c1`: cognitive coefficient (own's position). Optional, by default 1.
  * `c2`: social coefficient (others' position). Optional, by default 1.

Returns a [`Result`](@ref).


In [18]:
results = PSO(logbook, michalewicz, population, 30);

The output was suppressed so that we can analyse each part of the result separately:

In [19]:
@show optimum(results)

@show optimizer(results)

@show iterations(results)
@show f_calls(results);

optimum(results) = -1.7987560672102798
optimizer(results) = [2.1935299266534836, 1.5760612400982805]
iterations(results) = 30
f_calls(results) = 1550


We can also take a look at the logbook's records and see how the statistics we calculated changed throughout the run:

In [20]:
for (i, I) in enumerate(logbook.records)
    print("it: $(i) with best_pos: $(I[3]) and avg_pos: $(I[1]) \n")
end

it: 1 with best_pos: -0.7504688680623768 and avg_pos: 0.027014191784156233 
it: 2 with best_pos: -0.7906765701486448 and avg_pos: -0.11592097787713364 
it: 3 with best_pos: -0.8012991501612791 and avg_pos: -0.17770241942233952 
it: 4 with best_pos: -0.7963864986732629 and avg_pos: -0.17023969225290894 
it: 5 with best_pos: -0.8726099514277835 and avg_pos: -0.2928504362404555 
it: 6 with best_pos: -1.1697700406204043 and avg_pos: -0.33670799217170105 
it: 7 with best_pos: -1.6221334762010642 and avg_pos: -0.3834145274427395 
it: 8 with best_pos: -0.9285497521300503 and avg_pos: -0.37119067568197167 
it: 9 with best_pos: -1.7520089205738312 and avg_pos: -0.42151516296756136 
it: 10 with best_pos: -1.587048845608525 and avg_pos: -0.3869242929292781 
it: 11 with best_pos: -1.771893190795443 and avg_pos: -0.4246700864565007 
it: 12 with best_pos: -1.7416264377070607 and avg_pos: -0.5136524380147719 
it: 13 with best_pos: -1.7871172991946955 and avg_pos: -0.4094174620962018 
it: 14 with best